In [156]:
import pandas as pd
import numpy as np
from collections import namedtuple

In [157]:
all_data_raw = pd.read_csv("./tumorDetails-xy-coordinates.csv")
result = pd.read_csv("./bb_results/comp4_det_test_cancer.txt", sep = " ", header=None, 
                     names = ["im_name", "confidence", "yolo_x_min", "yolo_y_min", "yolo_x_max", "yolo_y_max"])
test = pd.read_csv("./test.txt", header=None)
all_data = all_data_raw.copy()
all_data.shape, result.shape

((2847, 8), (66041, 6))

In [158]:
all_data["im_name"] = [x.rsplit(sep="/",maxsplit=1)[-1] for x in all_data.image_name]
all_data["im_name"] = [x.rsplit(sep=".",maxsplit=1)[0] for x in all_data.im_name]
all_data["im_name"] = [x.replace(".", "_") for x in all_data.im_name]
#all_data.drop(["image_name"], axis=1, inplace=True)
all_data.head(3)

,image_name,tumor_location_x,tumor_location_y,tumor_radius,x_min,x_max,y_min,y_max,im_name
0,/Users/abqader/Desktop/imagesOnePlay/benign_wi...,956.0,2168.0,78.461102,878,1034,2090,2246,B_3159_1_LEFT_MLO
1,/Users/abqader/Desktop/imagesOnePlay/benign_wi...,484.0,2276.0,62.899176,421,547,2213,2339,B_3160_1_LEFT_CC
2,/Users/abqader/Desktop/imagesOnePlay/benign_wi...,868.0,3164.0,80.839964,787,949,3083,3245,B_3160_1_LEFT_MLO


In [159]:
result.head(3)

,im_name,confidence,yolo_x_min,yolo_y_min,yolo_x_max,yolo_y_max
0,A_1620_1_RIGHT_CC,0.005978,1.000000,1.000000,-328.081360,236.653442
1,A_1620_1_RIGHT_CC,0.009534,1.000000,1.000000,-154.828156,219.696747
2,A_1620_1_RIGHT_CC,0.006515,818.806702,1.507446,1017.546936,225.948425


In [160]:
# only take predictions with highest probability per image, in the case of multiple bb predictions
max_probs_ix = result.groupby("im_name").confidence.idxmax()
max_results = result.loc[max_probs_ix]

In [161]:
max_results.shape, all_data.shape

((448, 6), (2847, 9))

In [162]:
# only take relevant ground truth bounding boxes for test
truth = all_data.loc[all_data.im_name.isin(max_results.im_name)]

In [163]:
truth.shape, max_results.shape

((448, 9), (448, 6))

In [164]:
truth.head(2)

,image_name,tumor_location_x,tumor_location_y,tumor_radius,x_min,x_max,y_min,y_max,im_name
1499,/Users/abqader/Desktop/imagesOnePlay/cancers/c...,1252.0,2772.0,393.017836,859,1645,2379,3165,C_0002_1_LEFT_CC
1507,/Users/abqader/Desktop/imagesOnePlay/cancers/c...,2108.0,1504.0,331.090061,1777,2439,1173,1835,C_0009_1_RIGHT_CC


In [165]:
max_results.head(2)

,im_name,confidence,yolo_x_min,yolo_y_min,yolo_x_max,yolo_y_max
12393,A_1000_1_LEFT_MLO,0.085492,2292.368408,432.668701,2402.981201,676.918335
54075,A_1001_1_LEFT_CC,0.024673,1649.535889,1201.850098,1681.000000,1436.681641


In [166]:
iou = pd.merge(truth, max_results, on = "im_name")
cols = ['im_name', 'x_min','x_max', 'y_min', 'y_max', 'yolo_x_min','yolo_y_min', 'yolo_x_max', 'yolo_y_max']
iou_df = iou[cols]
iou_df.head()

,im_name,x_min,x_max,y_min,y_max,yolo_x_min,yolo_y_min,yolo_x_max,yolo_y_max
0,C_0002_1_LEFT_CC,859,1645,2379,3165,2424.156738,3489.210205,2729.819336,4176.296875
1,C_0009_1_RIGHT_CC,1777,2439,1173,1835,903.104553,1906.790527,1009.807068,2160.855713
2,C_0010_1_RIGHT_CC,2187,2677,2223,2713,242.550690,539.320862,345.972565,771.383606
3,C_0012_1_RIGHT_CC,2348,2660,1164,1476,2515.016602,4499.387207,2656.000000,4640.000000
4,C_0019_1_LEFT_CC,199,681,1263,1745,2076.646484,5610.426758,2416.547363,5758.567383


In [167]:
# compute IoU 
# area of overlap/area of union
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = (xB - xA + 1) * (yB - yA + 1)
    if interArea < 0: interArea=0

    # compute the area of the predicted and ground-truth bounding boxes
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    if interArea >= (boxAArea + boxBArea): interArea=0

    # compute the intersection over union by dividing the intersection area
    # by the sum of prediction + ground-truth areas minus the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    if iou < 0: return 0
    if iou > 1: return 0
    else: return iou # return the intersection over union value

In [168]:
iou=[]
for i in range(len(iou_df)):
    boxA = list(iou_df[['x_min', 'y_min','x_max','y_max']].loc[i])
    boxB = list(iou_df[['yolo_x_min','yolo_y_min', 'yolo_x_max', 'yolo_y_max']].loc[i])
    iou.append(bb_intersection_over_union(boxA, boxB))

In [169]:
iou_df["iou"] = iou
iou_df

,im_name,x_min,x_max,y_min,y_max,yolo_x_min,yolo_y_min,yolo_x_max,yolo_y_max,iou
0,C_0002_1_LEFT_CC,859,1645,2379,3165,2424.156738,3489.210205,2729.819336,4176.296875,0.434480
1,C_0009_1_RIGHT_CC,1777,2439,1173,1835,903.104553,1906.790527,1009.807068,2160.855713,0.131393
2,C_0010_1_RIGHT_CC,2187,2677,2223,2713,242.550690,539.320862,345.972565,771.383606,0.000000
3,C_0012_1_RIGHT_CC,2348,2660,1164,1476,2515.016602,4499.387207,2656.000000,4640.000000,0.000000
4,C_0019_1_LEFT_CC,199,681,1263,1745,2076.646484,5610.426758,2416.547363,5758.567383,0.000000
5,C_0020_1_LEFT_CC,-50,1666,1378,3094,3440.308350,4918.483398,3571.574951,5205.625000,0.000000
6,C_0020_1_LEFT_MLO,430,2146,1182,2898,112.844543,5098.237305,287.856537,5476.297852,0.114779
7,B_3003_1_RIGHT_CC,1834,2422,1738,2326,185.632233,896.325928,237.835968,1048.430298,0.000000
8,B_3017_1_LEFT_MLO,584,888,2852,3156,581.152100,5707.214844,843.631226,5848.000000,0.000000
9,B_3021_1_LEFT_CC,2008,2280,1792,2064,2790.385498,4150.798828,2897.461670,4400.690430,0.000000


In [141]:
# IOU FROM FIRST RUN
# test image h,w = 416

iou_df.iou.describe()

count    448.000000
mean       0.052839
std        0.150203
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.976369
Name: iou, dtype: float64

In [66]:
to_viz = iou_df.loc[iou_df.iou>1]
to_viz.shape

In [120]:
to_viz_out = to_viz.merge(all_data[["image_name", "im_name"]])

In [38]:
iou_df.to_csv("./bb_results/first_run.csv")

In [155]:
# h,w = 832

iou_df.iou.describe()

count    448.000000
mean       0.049828
std        0.141468
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.887157
Name: iou, dtype: float64

In [87]:
iou_df.to_csv("./bb_results/second_run.csv")

to_viz_run2 = iou_df.loc[iou_df.iou>1]
to_viz_run2.shape

(41, 10)

In [170]:
# h,w = 1248

iou_df.iou.describe()

count    448.000000
mean       0.043643
std        0.153892
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.999996
Name: iou, dtype: float64

In [177]:
viz_more = iou_df.sort_values(by=["iou"], ascending=False)
sum(viz_more.iou > 0.5)

16


###############################################################

In [128]:
test = pd.read_csv("./test.txt", header=None, names=["test"])
test.test = [x.rsplit(sep="/",maxsplit=1)[-1] for x in test.test]
test["img"] = [x.rsplit(sep=".",maxsplit=-1)[0] for x in test.test]

In [145]:
sum(test.img.duplicated(keep=False)), len(test)

(102, 448)

In [150]:
test_dupl = test.loc[test.img.duplicated(keep=False)]
test_dupl.sort_index(by="img")

//anaconda/envs/opencv/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  from ipykernel import kernelapp as app


,test,img
22,A_1006_1.LEFT_CC.jpg,A_1006_1
255,A_1006_1.LEFT_MLO.jpg,A_1006_1
161,A_1011_1.LEFT_MLO.jpg,A_1011_1
331,A_1011_1.LEFT_CC.jpg,A_1011_1
175,A_1022_1.LEFT_MLO.jpg,A_1022_1
272,A_1022_1.LEFT_CC.jpg,A_1022_1
268,A_1026_1.LEFT_CC.jpg,A_1026_1
228,A_1026_1.LEFT_MLO.jpg,A_1026_1
399,A_1043_1.LEFT_MLO.jpg,A_1043_1
44,A_1043_1.LEFT_CC.jpg,A_1043_1


In [151]:
test

,test,img
0,A_1620_1.RIGHT_CC.jpg,A_1620_1
1,B_3378_1.RIGHT_CC.jpg,B_3378_1
2,A_1720_1.LEFT_CC.jpg,A_1720_1
3,A_1142_1.LEFT_MLO.jpg,A_1142_1
4,B_3064_1.LEFT_MLO.jpg,B_3064_1
5,A_1558_1.LEFT_CC.jpg,A_1558_1
6,C_0008_1.RIGHT_CC.jpg,C_0008_1
7,C_0102_1.LEFT_MLO.jpg,C_0102_1
8,B_3498_1.LEFT_CC.jpg,B_3498_1
9,A_1985_1.RIGHT_CC.jpg,A_1985_1
